<a href="https://colab.research.google.com/github/eze-hong/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EC%B6%9C%EC%84%9D%EA%B3%BC%EC%A0%9C_%ED%99%8D%EC%9D%B4%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-2

1.   Python을 사용하여 API를 호출하는 코드를 작성
2.   2015년 1월부터 2024년 12월까지의 개인 유형의 현년 전기, 가스, 수도, 지역난방 에너지 사용량 데이터를 수집하는 프로그램을 작성
3. API 호출 성공을 확인할 수 있는 실행 결과를 캡처하여 첨부


In [4]:
import requests
import json
import pandas as pd
import numpy as np

url = 'http://openapi.seoul.go.kr:8088/626b6c4a5873656f36336e756f4b6f/json/energyUseDataSummaryInfo/1/5/'

params = {
    'KEY' : '626b6c4a5873656f36336e756f4b6f',
    'TYPE' : 'json',
    'SERVICE': 'energyUseDataSummaryInfo',
    'START_INDEX' : '1',
    'END_INDEX' : '100',
}
## 데이터 수집
response = requests.get(url, params=params)

data = response.json()['energyUseDataSummaryInfo'] ##['row']
##df = pd.DataFrame(data)
print(data)
##print(data['RESULT']['CODE'])

{'list_total_count': 1241, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2025', 'MON': '06', 'MM_TYPE': '개인', 'CNT': '1269454', 'EUS': '278745288', 'EUS1': '275851011', 'EUS2': '273151435', 'ECO2_1': '4244065', 'ECO2_2': '2076623.656', 'GUS': '13084768', 'GUS1': '11889039', 'GUS2': '12094650', 'GCO2_1': '1091102', 'GCO2_2': '2182204', 'WUS': '14927683.6', 'WUS1': '15166284.8', 'WUS2': '15371662.2', 'WCO2_1': '-341289.9', 'WCO2_2': '-611822.446', 'HUS': '379182.22', 'HUS1': '368839.049', 'HUS2': '389847.167', 'HCO2_1': '-160.888', 'HCO2_2': '-22448.67848156', 'REG_DATE': '2025-09-01 04:02:15.0'}, {'YEAR': '2025', 'MON': '06', 'MM_TYPE': '학교', 'CNT': '6745', 'EUS': '117808793', 'EUS1': '114793127', 'EUS2': '114623246', 'ECO2_1': '3100606.5', 'ECO2_2': '3096973.668', 'GUS': '2093441', 'GUS1': '2046804', 'GUS2': '2061648', 'GCO2_1': '39207', 'GCO2_2': '78414', 'WUS': '1824338.2', 'WUS1': '1914804.2', 'WUS2': '1956560.2', 'WCO2_1': '-111344', 'WCO2_2': '-333633.

In [11]:
import requests
import json
import pandas as pd
import numpy as np

url = 'http://openapi.seoul.go.kr:8088/626b6c4a5873656f36336e756f4b6f/json/energyUseDataSummaryInfo/1/5/'

params = {
    'KEY' : '626b6c4a5873656f36336e756f4b6f',
    'TYPE' : 'json',
    'SERVICE': 'energyUseDataSummaryInfo',
    'START_INDEX' : '1',
    'END_INDEX' : '100',
}
## 데이터 수집
##response = requests.get(url, params=params)

##data = response.json()['energyUseDataSummaryInfo']['row']
##print(json.dumps(data, indent=4, ensure_ascii=False))

##df = pd.DataFrame(data)
##print(df)

##개인타입인지 확인
##is_개인 = df['MM_TYPE'] == '개인'

##확인한 결과로 필터링한 데이터
##개인df = df[is_개인]
##print(개인df)
##EUS전기
##selected_col = 개인df[['YEAR']['MON']['MM_TYPE']['EUS']['GUS']['WUS']['HUS']]
##selected_col = 개인df[['YEAR','MON','MM_TYPE','EUS','GUS','WUS','HUS']]
##print(selected_col)


def get_year_months():
  year_months = []
  for year in range(2015, 2016+1):
    ##for year in range(2015, 2024+1):
    for month in range(1,12+1):
      year_months.append(f'{year}{month:02d}')
  return year_months

##print(get_year_months())
responses = []

for year_month in get_year_months():
  # 데이터를 수집하고 responses에 추가
  ##print(year_month[:4])
    year = year_month[:4]
    mon = year_month[4:]
    url = f'http://openapi.seoul.go.kr:8088/626b6c4a5873656f36336e756f4b6f/json/energyUseDataSummaryInfo/1/5/{year}/{mon}'
    response = requests.get(url, params=params)
    data = response.json()['energyUseDataSummaryInfo']
    if data['RESULT']['CODE'] == 'INFO-000' :
        rows =  data['row']
        개인_row = [r for r in rows if r['MM_TYPE'] == '개인']

        if(len(개인_row)!=0) :
            responses.append(개인_row)
    ##print(개인_row)
    ##df = pd.DataFrame(data)
    ##개인df = df[df['MM_TYPE'] == '개인'][['YEAR','MON','MM_TYPE','EUS','GUS','WUS','HUS']]
    ##print(개인df)
    ##responses.append(개인df)

print(responses)


[[{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0'}], [{'YEAR': '2015', 'MON': '02', 'MM_TYPE': '개인', 'CNT': '774620', 'EUS': '189974230', 'EUS1': '193611430', 'EUS2': '200055533', 'ECO2_1': '-6859251.5', 'ECO2_2': '-2908322.636', 'GUS': '56487358', 'GUS1': '59353536', 'GUS2': '66191173', 'GCO2_1': '-6284996.5', 'GCO2_2': '-14078392.16', 'WUS': '12656888.218', 'WUS1': '12713146.172', 'WUS2': '12948410.081', 'WCO2_1': '-173889.9085', 'WCO2_2': '-57731.449622', 'HUS': '18793320.213', 'H